In [ ]:
!pip install gensim
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install --upgrade language-check
!pip install pycontractions
!pip install Unidecode
!pip install word2number

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_265"
OpenJDK Runtime Environment (build 1.8.0_265-8u265-b01-0ubuntu2~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.265-b01, mixed mode)
  Created wheel for language-check: filename=language_check-1.1-cp36-none-any.whl size=90190900 sha256=72dc884bfbc9d86fc08a899f94a3ac058701b06d84f2ff2091fa1bd77197ff2c
  Stored in directory: /root/.cache/pip/wheels/d5/46/82/90a89c23eac1837364ed7217a9eed71bc9e6ad4825be93968e
Successfully built language-check
     |████████████████████████████████| 245kB 2.7MB/s 
  Created wheel for word2number: filename=word2number-1.1-cp36-none-any.whl size=5588 sha256=b0b7e0183821cac330bc179d18a1e320a79ffbf310c3f6c39bce7037cd0ff29c
  Stored in directory: /root/.cache/pip/wheels/46/2f/53/5f5c1d275492f2fce1cdab9a9bb12d49286dead829a4078e0e
Successfully built word2number


In [ ]:
import pandas as pd
import numpy as np
import logging
import sys
from time import time
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
import gensim.downloader as api
from gensim.models import Word2Vec
word2vec = api.load('word2vec-google-news-300')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[==================================================] 100.0% 1662.8/1662.8MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
data_directory = "drive/My Drive/Projects/TextSummarizer/data/"

In [ ]:
input_path = data_directory + "to_summarize.txt"  #remove ‘content/’ from path then use 
input_file = open(input_path, "r")
input_data = input_file.read()

In [ ]:
print(input_data)

﻿Cambodia's two-party opposition asked the Asian Development Bank Monday to stop providing loans to the incumbent government, which it calls illegal. Negotiations to form the next government have become deadlocked, and opposition party leaders Prince Norodom Ranariddh and Sam Rainsy are out of the country following threats of arrest from strongman Hun Sen. Hun Sen complained Monday that the opposition was trying to make its members' return an international issue. Hun Sen's ruling party narrowly won a majority in elections in July, but the opposition _ claiming widespread intimidation and fraud _ has denied Hun Sen the two-thirds vote in parliament required to approve the next government. Meanwhile, it says, the old government is holding power illegally. Ranariddh and Sam Rainsy renewed their international lobbying campaign against the old government Monday in a letter to ADB President Mitsuo Sato calling for the bank to stop lending money to it. ``We respectfully advise the Asian Devel

https://github.com/sdimi/average-word2vec

In [ ]:
def avgEmbed(x):
  #print(x)
  count = 0
  allvecs = []

  for i in sent_tokenize(x): 
    temp = [] 
      
    # tokenize the sentence into words 
    for j in word_tokenize(i): 
        if j in word2vec.vocab:
          allvecs.append(word2vec[j]) 
  allvecs = np.array(allvecs)
  avg_vec = np.mean(allvecs, axis=0)
  return avg_vec

In [ ]:
centroid_embedding = avgEmbed(input_data)
print("centroid embedding")
print(centroid_embedding)
print()

centroid embedding
[ 0.01869853  0.06292582  0.06161173  0.03106331 -0.05463019 -0.07770131
  0.00263508 -0.08327677  0.08296008  0.06438486 -0.03158617 -0.06306413
 -0.02830565  0.0678649  -0.11372563  0.04595166  0.05240729  0.06050681
 -0.01225189 -0.04943531  0.00328563  0.02809335  0.02078725 -0.00133217
  0.0405976  -0.04758782 -0.04896661  0.00807588  0.04299694  0.00847158
  0.00337915 -0.04487269 -0.04657102  0.04130132  0.02836825  0.0003048
  0.00248095  0.01660331  0.01312113  0.01442963  0.03831381 -0.06367771
  0.1115034  -0.01442335 -0.04121671 -0.04220043 -0.05493278  0.03350885
 -0.01318608  0.04669963  0.03324495  0.02512429 -0.01421917 -0.00064877
 -0.03891816  0.05784785 -0.07771877 -0.05435426  0.01063848 -0.08135227
 -0.04423559  0.09048653 -0.05591033 -0.04696152 -0.01769464 -0.00975741
  0.01079938  0.08900593  0.01451333  0.08785807  0.03963256  0.02858258
  0.07670301  0.03653375 -0.07222239 -0.05686497  0.07227369  0.07543328
  0.07816242  0.08086504  0.01498

In [ ]:
sentences = []
sentences_embeddings = []

for sent in sent_tokenize(input_data):
  sentences.append(sent)
  sentences_embeddings.append(avgEmbed(sent))

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
record = []
for i in range(len(sentences_embeddings)):
  similarity = (1 - cosine(centroid_embedding, sentences_embeddings[i]))
  record.append((i, similarity))
rank = list(reversed(sorted(record, key=lambda tup: tup[1])))

In [ ]:
sentence_ids = []
summary_char_num = 0
stop = False
i = 0
redundancy_threshold = 0.95
summary_limit = 500

while not stop and i < len(rank):
  sentence_id = rank[i][0]
  new_vector = sentences_embeddings[sentence_id]
  sent_char_num = len(sentences[sentence_id])
  redundancy = [sentences_embeddings[k] for k in sentence_ids
                if (1 - cosine(new_vector, sentences_embeddings[k]) > redundancy_threshold)]

  if not redundancy:
      summary_char_num += sent_char_num
      sentence_ids.append(sentence_id)
  i += 1

  if summary_char_num > summary_limit:
      stop = True

In [ ]:
sentence_ids = sorted(sentence_ids)
result_list = map(lambda sent_id: sentences[sent_id], sentence_ids)

# Format output
summary = " ".join(result_list)
print(summary)

Negotiations to form the next government have become deadlocked, and opposition party leaders Prince Norodom Ranariddh and Sam Rainsy are out of the country following threats of arrest from strongman Hun Sen. Hun Sen complained Monday that the opposition was trying to make its members' return an international issue. Worried that party colleagues still face arrest for their politics, opposition leader Sam Rainsy sought further clarification Friday of security guarantees promised by strongman Hun Sen. Sam Rainsy wrote in a letter to King Norodom Sihanouk that he was eager to attend the first session of the new National Assembly on Nov. 25, but complained that Hun Sen's assurances were not strong enough to ease concerns his party members may be arrested upon their return to Cambodia.


In [ ]:
for sent in sent_tokenize(summary):
  print(sent)

Negotiations to form the next government have become deadlocked, and opposition party leaders Prince Norodom Ranariddh and Sam Rainsy are out of the country following threats of arrest from strongman Hun Sen. Hun Sen complained Monday that the opposition was trying to make its members' return an international issue.
Worried that party colleagues still face arrest for their politics, opposition leader Sam Rainsy sought further clarification Friday of security guarantees promised by strongman Hun Sen. Sam Rainsy wrote in a letter to King Norodom Sihanouk that he was eager to attend the first session of the new National Assembly on Nov. 25, but complained that Hun Sen's assurances were not strong enough to ease concerns his party members may be arrested upon their return to Cambodia.


In [ ]:
summary_file = open(data_directory + "summary.txt", "w")
for sent in sent_tokenize(summary):
  summary_file.write(sent)
summary_file.close()